In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - plot_fit_gse.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Plot fits to the GS/E sample Gaia/APOGEE data
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, warnings, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt
import corner

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil
from ges_mass import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.8]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 2000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

## Convenience functions

In [ ]:
def make_corner_with_ml_params(hf):
    '''Convenience function to make a corner plot with the optimized
    (initial and post) parameters, the ML sample from the chain, and the
    median parameters from the chain.
    '''
    fig = pplot.plot_corner(hf,**corner_kwargs)

    # Add the best-fitting model
    labels = ['Init Optimization','Post Optimization','MCMC Max(L)',
              'MCMC Medians']
    colors = ['DarkGreen','DodgerBlue','Crimson','Black']
    # linestyles = ['-','--','-.',':']
    params = [hf.get_ml_params('init'),
            hf.get_ml_params('post'),
            hf.get_ml_params('mcmc_ml'),
            hf.get_ml_params('mcmc_median'),]
    for i,pm in enumerate(params):
        params[i] = pdens.denormalize_parameters(pm, hf.densfunc).flatten()
    
    for i in range(len(params)):
        overplot_lines_kwargs = {'color':colors[i],
                                'label':labels[i],
                                'alpha':1.,
                                'lw':2.}
        corner.overplot_lines(fig, params[i], **overplot_lines_kwargs)
    
    # Legend axis
    axl = fig.add_axes([0.7,0.7,0.1,0.1])
    axl.axis('off')
    for i in range(len(params)):
        axl.plot(0,0,color=colors[i],label=labels[i])
    axl.legend(loc='center',frameon=False,fontsize=16)
    
    return fig

## Plotting parameters

In [ ]:
corner_kwargs={'samples':None,
               'plot_mass':False,
               'thin':None,
               'thin_to':int(1e5),
               'quantiles':[0.16,0.5,0.84],
               'show_titles':True,
               'truths':None,
               'corner_kwargs':{}}

## Plot the best-fitting density profile for each GS/E sample

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_gse)
version = '100w_2e3n'
fit_type = 'gse'

### e-Lz selection
Best density profile is `SC+D`

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_gse)
version = '100w_2e3n'
fit_type = 'gse'
selec = ['eLz',]
densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
def usr_log_prior(densfunc,params):
    return None
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
hf.get_results()
hf.get_loglike_ml_aic_bic()

In [ ]:
fig = make_corner_with_ml_params(hf)

### AD selection
Best density profile is `SC`

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_gse)
version = '100w_2e3n'
fit_type = 'gse'
selec = ['AD',]
densfunc = pdens.triaxial_single_cutoff_zvecpa
def usr_log_prior(densfunc,params):
    return None
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
hf.get_results()
hf.get_loglike_ml_aic_bic()

In [ ]:
fig = make_corner_with_ml_params(hf)

### JRLz selection
Best density profile is `SC+D`

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_gse)
version = '100w_2e3n'
fit_type = 'gse'
selec = ['JRLz',]
densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
def usr_log_prior(densfunc,params):
    return None
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
hf.get_results()
hf.get_loglike_ml_aic_bic()

In [ ]:
fig = make_corner_with_ml_params(hf)

### Plot the best-fitting density profile for the whole sample
Best is `SPL+D`

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_all)
version = '100w_2e3n'
fit_type = 'all'
selec = None
densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
def usr_log_prior(densfunc,params):
    return None
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
hf.get_results()
hf.get_loglike_ml_aic_bic()

In [ ]:
fig = make_corner_with_ml_params(hf)

## Now examine the 1D distributions of parameters, and look for ways to organize them by likelihood

In [ ]:
feh_range_fit = copy.deepcopy(feh_range_gse)
version = '100w_2e3n'
fit_type = 'gse'
selec = ['AD',]
densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
def usr_log_prior(densfunc,params):
    return None
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
hf.get_results()
hf.get_loglike_ml_aic_bic()

### Histogram of parameters

In [ ]:
fig = plt.figure(figsize=(20,8))
axs = fig.subplots(nrows=2,ncols=4).flatten()

# Plot the data
samples = hf.samples
samples = pdens.denormalize_parameters(samples, hf.densfunc)

loglike = hf.loglike
n_samples,n_params = samples.shape
param_names = pdens.get_densfunc_mcmc_labels(hf.densfunc, physical_units=True)
labels = ['Init Optimization','Post Optimization','MCMC Max(L)',
              'MCMC Medians']
colors = ['DarkGreen','DodgerBlue','Crimson','Black']
params = [hf.get_ml_params('init'),
          hf.get_ml_params('post'),
          hf.get_ml_params('mcmc_ml'),
          hf.get_ml_params('mcmc_median'),]
for i,pm in enumerate(params):
    params[i] = pdens.denormalize_parameters(pm, hf.densfunc).flatten()

for i in range(8):
    if i >= n_params:
        axs[i].axis('off')
        continue
    axs[i].hist(samples[:,i],histtype='step',bins=20,color='Black')
    for j,p in enumerate(params):
        axs[i].axvline(p[i],color=colors[j],lw=2,label=labels[j])
    if i == 0:
        axs[i].legend(loc='upper left',fontsize=8)
    if i in [0,4]:
        axs[i].set_ylabel('N')
    axs[i].set_xlabel(param_names[i])

#fig.subplots_adjust()
fig.show()

Parameters indexed by likelihood